In [7]:
# Cell 1: Install packages
!pip install gradio PyPDF2 plotly openai

# Cell 2: Paste the entire code from the artifact

In [ ]:
import gradio as gr
import PyPDF2
import re
from io import BytesIO
import plotly.graph_objects as go
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Initialize models (will be loaded once)
summarizer = None
text_generator = None

def load_models():
    """Load Hugging Face models - runs once at startup"""
    global summarizer, text_generator

    try:
        print("Loading AI models... This may take a minute on first run.")

        # Use a smaller, faster model for Colab
        model_name = "facebook/bart-large-cnn"  # Good for summarization
        summarizer = pipeline("summarization", model=model_name, device=-1)  # CPU

        # For text generation (resume enhancement)
        gen_model_name = "gpt2"  # Lightweight and fast
        text_generator = pipeline("text-generation", model=gen_model_name, device=-1)

        print("✅ Models loaded successfully!")
        return True
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")
        return False

def extract_text_from_pdf(pdf_file):
    """Extract text from PDF file"""
    try:
        if isinstance(pdf_file, bytes):
            pdf_file = BytesIO(pdf_file)

        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text, len(pdf_reader.pages)
    except Exception as e:
        return f"Error reading PDF: {str(e)}", 0

def analyze_document_synopsis(pdf_file, text):
    """Analyze document basic information"""
    if isinstance(pdf_file, bytes):
        file_size = len(pdf_file) / 1024
        pdf_stream = BytesIO(pdf_file)
    else:
        file_size = len(pdf_file.getvalue()) / 1024
        pdf_stream = pdf_file

    _, page_count = extract_text_from_pdf(pdf_stream)
    word_count = len(text.split())

    ats_score = 100
    if page_count > 2:
        ats_score -= 15
    if file_size > 1024:
        ats_score -= 10

    return {
        "ATS Compliance": f"{ats_score}%",
        "File Type": "PDF",
        "File Size": f"{file_size:.2f} KB",
        "Page Count": page_count,
        "Word Count": word_count
    }

def analyze_data_identification(text):
    """Identify key data points in resume"""
    phone_pattern = r'\b(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b'
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    linkedin_pattern = r'(linkedin\.com/in/[A-Za-z0-9_-]+|linkedin\.com/pub/[A-Za-z0-9_-]+)'

    phone = re.findall(phone_pattern, text)
    email = re.findall(email_pattern, text)
    linkedin = re.findall(linkedin_pattern, text, re.IGNORECASE)

    education = bool(re.search(r'\b(education|degree|university|college|bachelor|master)\b', text, re.IGNORECASE))
    work_history = bool(re.search(r'\b(experience|work history|employment|position|job)\b', text, re.IGNORECASE))
    skills = bool(re.search(r'\b(skills|competencies|expertise|proficient)\b', text, re.IGNORECASE))

    date_patterns = [
        r'\b\d{4}\s*-\s*\d{4}\b',
        r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{4}\b',
        r'\b\d{1,2}/\d{4}\b'
    ]
    dates_found = any(re.search(pattern, text, re.IGNORECASE) for pattern in date_patterns)

    return {
        "Phone Number": "✓ Found" if phone else "✗ Missing",
        "E-mail Address": "✓ Found" if email else "✗ Missing",
        "LinkedIn URL": "✓ Found" if linkedin else "✗ Missing",
        "Education": "✓ Found" if education else "✗ Missing",
        "Work History": "✓ Found" if work_history else "✗ Missing",
        "Skills / Achievements": "✓ Found" if skills else "✗ Missing",
        "Date Formatting": "✓ Consistent" if dates_found else "✗ Inconsistent"
    }

def analyze_lexical(text):
    """Perform lexical analysis"""
    words = text.split()
    total_words = len(words)

    pronouns = r'\b(I|me|my|mine|we|us|our|ours)\b'
    pronoun_count = len(re.findall(pronouns, text, re.IGNORECASE))
    pronoun_percentage = (pronoun_count / total_words * 100) if total_words > 0 else 0

    numbers = re.findall(r'\b\d+[%]?\b', text)
    numeric_percentage = (len(numbers) / total_words * 100) if total_words > 0 else 0

    unique_words = len(set(word.lower() for word in words if word.isalpha()))
    vocab_ratio = (unique_words / total_words * 100) if total_words > 0 else 0

    avg_word_length = sum(len(word) for word in words if word.isalpha()) / len([w for w in words if w.isalpha()]) if words else 0
    reading_level = "Advanced" if avg_word_length > 6 else "Intermediate" if avg_word_length > 4 else "Basic"

    power_words = ['achieved', 'improved', 'increased', 'developed', 'managed', 'led', 'created', 'designed', 'implemented']
    power_word_count = sum(1 for word in power_words if word in text.lower())

    return {
        "Personal Pronouns": f"{pronoun_percentage:.1f}% (Lower is better)",
        "Numericized Data": f"{numeric_percentage:.1f}% ({len(numbers)} metrics found)",
        "Vocabulary Level": f"{vocab_ratio:.1f}% unique words",
        "Reading Level": reading_level,
        "Power Words": f"{power_word_count} action words found"
    }

def analyze_semantic(text):
    """Perform semantic analysis"""
    words = text.split()
    total_words = len(words)

    achievement_pattern = r'\b\d+[%]?\s*(increase|decrease|improve|reduce|save|generate|grow|boost)\w*\b'
    achievements = re.findall(achievement_pattern, text, re.IGNORECASE)
    achievement_count = len(achievements)

    common_hard_skills = [
        'python', 'java', 'javascript', 'sql', 'aws', 'azure', 'docker', 'kubernetes',
        'machine learning', 'data analysis', 'project management', 'agile', 'scrum',
        'excel', 'powerpoint', 'salesforce', 'tableau', 'react', 'node.js', 'git',
        'html', 'css', 'mongodb', 'postgresql', 'tensorflow', 'pytorch', 'spark'
    ]
    hard_skills_found = [skill for skill in common_hard_skills if skill.lower() in text.lower()]
    hard_skills_count = len(hard_skills_found)

    skills_ratio = (hard_skills_count / (total_words / 100)) if total_words > 0 else 0

    return {
        "Measurable Achievements": f"{achievement_count} quantified results",
        "Hard Skills Identified": f"{hard_skills_count} technical skills",
        "Skills Efficiency Ratio": f"{skills_ratio:.2f}",
        "Top Skills": ", ".join(hard_skills_found[:5]) if hard_skills_found else "None detected"
    }

def calculate_ats_score(synopsis, identification, lexical, semantic):
    """Calculate overall ATS score"""
    score = 0

    if int(synopsis["Page Count"]) <= 2:
        score += 10
    if int(synopsis["Word Count"]) >= 300:
        score += 10

    identification_checks = [v for v in identification.values() if "✓" in str(v)]
    score += len(identification_checks) * 4.3

    pronoun_val = float(lexical["Personal Pronouns"].split("%")[0])
    if pronoun_val < 2:
        score += 10
    numeric_val = float(lexical["Numericized Data"].split("%")[0])
    if numeric_val > 5:
        score += 10
    if lexical["Reading Level"] in ["Intermediate", "Advanced"]:
        score += 5

    achievement_count = int(semantic["Measurable Achievements"].split()[0])
    if achievement_count >= 3:
        score += 10
    skills_count = int(semantic["Hard Skills Identified"].split()[0])
    if skills_count >= 5:
        score += 10
    if skills_count >= 10:
        score += 5

    return min(int(score), 100)

def get_ai_analysis(text, synopsis, identification, lexical, semantic):
    """Generate AI analysis using rule-based insights + AI enhancement"""

    # Rule-based analysis
    issues = []
    strengths = []

    # Check issues
    if int(synopsis["Page Count"]) > 2:
        issues.append("Resume is longer than 2 pages - consider condensing")

    if "✗" in identification["Phone Number"]:
        issues.append("Missing phone number - add contact information")
    if "✗" in identification["E-mail Address"]:
        issues.append("Missing email address - essential for contact")
    if "✗" in identification["LinkedIn URL"]:
        issues.append("Missing LinkedIn URL - highly recommended for networking")

    pronoun_val = float(lexical["Personal Pronouns"].split("%")[0])
    if pronoun_val > 5:
        issues.append(f"Too many personal pronouns ({pronoun_val:.1f}%) - use third person or remove them")

    numeric_val = float(lexical["Numericized Data"].split("%")[0])
    if numeric_val < 5:
        issues.append("Lack of quantifiable achievements - add numbers, percentages, metrics")

    achievement_count = int(semantic["Measurable Achievements"].split()[0])
    if achievement_count < 3:
        issues.append("Few measurable achievements - quantify your impact with specific results")

    skills_count = int(semantic["Hard Skills Identified"].split()[0])
    if skills_count < 5:
        issues.append("Limited technical skills listed - add relevant hard skills for your field")

    # Check strengths
    if int(synopsis["Page Count"]) <= 2:
        strengths.append("Good length - concise and focused")

    if numeric_val > 10:
        strengths.append("Strong use of metrics and quantifiable data")

    if achievement_count >= 5:
        strengths.append("Excellent track record of measurable achievements")

    if skills_count >= 10:
        strengths.append("Comprehensive technical skills section")

    if pronoun_val < 2:
        strengths.append("Professional tone - minimal use of personal pronouns")

    # Generate analysis text
    analysis = f"""## 🎯 Overall Assessment

{"Your resume shows strong fundamentals with good ATS optimization." if len(strengths) > len(issues) else "Your resume needs improvement in several key areas for better ATS performance."}

## ✅ Key Strengths ({len(strengths)})
"""

    for i, strength in enumerate(strengths[:5], 1):
        analysis += f"{i}. {strength}\n"

    if not strengths:
        analysis += "- Consider implementing the improvements below to strengthen your resume\n"

    analysis += f"""
## ⚠️ Areas for Improvement ({len(issues)})
"""

    for i, issue in enumerate(issues[:5], 1):
        analysis += f"{i}. {issue}\n"

    analysis += """
## 🚀 ATS Optimization Tips

1. **Use Action Verbs**: Start bullet points with powerful verbs (Achieved, Developed, Led, Implemented)
2. **Quantify Everything**: Add numbers wherever possible (%, $, time saved, team size)
3. **Keywords Matter**: Include industry-specific terms and skills from job descriptions
4. **Format Simply**: Use standard fonts, avoid tables/graphics that ATS can't read
5. **Customize**: Tailor your resume for each job application

## 💡 Quick Wins

- Add 3-5 more quantifiable achievements to your bullet points
- Include specific metrics (increased by X%, reduced by Y hours)
- List technical skills relevant to your target role
- Ensure all contact information is present and formatted correctly
"""

    return analysis

def get_enhanced_bullets(text):
    """Generate enhanced bullet point examples"""

    # Extract existing bullet points
    bullet_pattern = r'[•\-\*]\s*(.+?)(?=\n[•\-\*]|\n\n|\Z)'
    bullets = re.findall(bullet_pattern, text, re.DOTALL)

    if not bullets:
        return """## ✨ Enhanced Bullet Point Examples

Since no bullet points were detected, here are examples of strong resume bullets:

**Weak**: Responsible for managing team
**Strong**: Led cross-functional team of 12 developers, reducing project delivery time by 30% and increasing customer satisfaction scores from 3.2 to 4.5/5

**Weak**: Worked on sales activities
**Strong**: Drove $2.5M in new revenue by identifying and closing 45+ enterprise accounts, exceeding quarterly quota by 140%

**Weak**: Improved system performance
**Strong**: Optimized database queries and API endpoints, reducing average response time from 2.5s to 400ms and improving user retention by 25%

### Tips for Writing Strong Bullets:
1. Start with an action verb (Led, Developed, Achieved, Increased)
2. Add specific numbers and metrics
3. Show impact and results, not just responsibilities
4. Use industry-specific keywords
"""

    enhancement = "## ✨ Enhanced Bullet Point Examples\n\n"
    enhancement += "Here are your bullets rewritten with stronger impact:\n\n"

    # Take first 3 bullets and enhance them
    action_verbs = ['Achieved', 'Developed', 'Led', 'Implemented', 'Increased', 'Reduced', 'Optimized', 'Drove']

    for i, bullet in enumerate(bullets[:3], 1):
        original = bullet.strip()[:100]  # Limit length

        enhancement += f"**Original {i}**: {original}\n\n"
        enhancement += f"**Enhanced {i}**: "

        # Add action verb if missing
        if not any(original.lower().startswith(verb.lower()) for verb in action_verbs):
            enhancement += f"Achieved {original.lower()}"
        else:
            enhancement += original

        enhancement += " [Add specific metrics: numbers, percentages, timeframes, or dollar amounts here]\n\n"

    enhancement += """
### Enhancement Tips:
- Add quantifiable results (increased by 40%, saved $50K, reduced time by 2 hours)
- Include scope (team size, budget, project duration)
- Show business impact (revenue, efficiency, customer satisfaction)
- Use strong action verbs at the start
"""

    return enhancement

def create_ats_chart(ats_score):
    """Create ATS score visualization"""
    remaining = 100 - ats_score
    colors = ['#10b981' if ats_score >= 70 else '#f59e0b' if ats_score >= 50 else '#ef4444', '#e5e7eb']

    fig = go.Figure(data=[go.Pie(
        labels=['ATS Score', 'Gap'],
        values=[ats_score, remaining],
        hole=.6,
        marker=dict(colors=colors),
        textinfo='none',
        hoverinfo='label+percent'
    )])

    fig.update_layout(
        title=dict(text=f"<b>ATS Score: {ats_score}/100</b>", x=0.5, xanchor='center', font=dict(size=24)),
        annotations=[dict(text=f'<b>{ats_score}</b><br>ATS Score', x=0.5, y=0.5, font_size=32, showarrow=False)],
        showlegend=True,
        height=400,
        margin=dict(t=80, b=20, l=20, r=20)
    )

    return fig

def format_results(data_dict):
    """Format results as HTML table"""
    html = "<table style='width:100%; border-collapse: collapse;'>"
    for key, value in data_dict.items():
        html += f"<tr style='border-bottom: 1px solid #ddd;'>"
        html += f"<td style='padding: 10px; font-weight: bold; width: 40%;'>{key}</td>"
        html += f"<td style='padding: 10px;'>{value}</td>"
        html += "</tr>"
    html += "</table>"
    return html

def analyze_resume(pdf_file):
    """Main analysis function"""
    if pdf_file is None:
        return "Please upload a PDF file", "", "", "", "", None, "", ""

    # Extract text
    text, _ = extract_text_from_pdf(pdf_file)

    if "Error" in text:
        return text, "", "", "", "", None, "", ""

    # Perform analyses
    synopsis = analyze_document_synopsis(pdf_file, text)
    identification = analyze_data_identification(text)
    lexical = analyze_lexical(text)
    semantic = analyze_semantic(text)

    # Calculate ATS score
    ats_score = calculate_ats_score(synopsis, identification, lexical, semantic)

    # Create chart
    chart = create_ats_chart(ats_score)

    # Get AI analysis (rule-based + insights)
    ai_analysis = get_ai_analysis(text, synopsis, identification, lexical, semantic)
    enhanced_bullets = get_enhanced_bullets(text)

    # Format results
    synopsis_html = format_results(synopsis)
    identification_html = format_results(identification)
    lexical_html = format_results(lexical)
    semantic_html = format_results(semantic)

    # Summary
    summary = f"""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; margin-bottom: 20px;'>
        <h2 style='margin: 0; font-size: 28px;'>📊 Overall ATS Score: {ats_score}/100</h2>
        <p style='margin: 10px 0 0 0; font-size: 16px;'>
            {'✅ Excellent! Your resume is ATS-optimized.' if ats_score >= 70 else
             '⚠️ Good, but needs improvement.' if ats_score >= 50 else
             '❌ Needs significant optimization.'}
        </p>
    </div>
    """

    return summary, synopsis_html, identification_html, lexical_html, semantic_html, chart, ai_analysis, enhanced_bullets

# Create Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="AI Resume Analyzer") as demo:
    gr.Markdown("""
    # 🎯 AI Resume Analyzer & ATS Score Calculator
    ##

    Upload your resume (PDF format) to get a comprehensive ATS analysis with AI-powered insights.
    This uses advanced rule-based AI analysis - completely free and runs locally!
    """)

    with gr.Row():
        with gr.Column(scale=1):
            pdf_input = gr.File(
                label="📄 Upload Your Resume (PDF)",
                file_types=[".pdf"],
                type="binary"
            )
            analyze_btn = gr.Button("🔍 Analyze Resume", variant="primary", size="lg")

    summary_output = gr.HTML(label="Summary")

    with gr.Row():
        with gr.Column():
            chart_output = gr.Plot(label="ATS Score Visualization")

    with gr.Tabs():
        with gr.Tab("🤖 AI Analysis & Recommendations"):
            ai_analysis_output = gr.Markdown(label="Detailed Analysis")

        with gr.Tab("✨ Enhanced Bullet Points"):
            enhanced_bullets_output = gr.Markdown(label="Before & After Examples")

        with gr.Tab("📋 Document Synopsis"):
            synopsis_output = gr.HTML()

        with gr.Tab("🔍 Data Identification"):
            identification_output = gr.HTML()

        with gr.Tab("📝 Lexical Analysis"):
            lexical_output = gr.HTML()

        with gr.Tab("🧠 Semantic Analysis"):
            semantic_output = gr.HTML()

    gr.Markdown("""
    ---
    ### 📊 Scoring Criteria:
    - **Document Synopsis**: File optimization, page count, word count
    - **Data Identification**: Contact info, sections completeness
    - **Lexical Analysis**: Professional language, quantified data
    - **Semantic Analysis**: Measurable achievements, hard skills

    ### 💡 Features:
    - ✅ 100% Free
    - ✅ AI-powered analysis and recommendations
    - ✅ ATS optimization tips
    - ✅ Enhanced bullet point examples
    - ✅ Comprehensive scoring across 4 dimensions
    """)

    analyze_btn.click(
        fn=analyze_resume,
        inputs=[pdf_input],
        outputs=[summary_output, synopsis_output, identification_output,
                lexical_output, semantic_output, chart_output,
                ai_analysis_output, enhanced_bullets_output]
    )

if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d709fd6f55e6e9a6b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",